In [1]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm

In [2]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("blog crawl.xlsx")        # 기본 모델
num_list = len(url_load)  

print(num_list)
url_load

100


,Unnamed: 0,url,title
0,0,https://srw.kr/205,"#88 맨유의 원조 킹, 데니스 로"
1,1,https://blog.naver.com/alth2214?Redirect=Log&l...,"스파이크리스 골프화, 맨유스페셜에디션 아디다스 아디크로스 레트로 솔직 후기"
2,2,https://blog.naver.com/lgw201?Redirect=Log&log...,"라울 히메네스, 그는 맨유에 어울리는 공격수일까?"
3,3,https://blog.naver.com/kung11?Redirect=Log&log...,맨유의 새로운 7번?... 제이든 산초의 시그니처 축구화 발매 예정?!!
4,4,https://z2inny.blog.me/222053307929,맨유를 떠나 인터밀란으로 이적하는 알렉시스 산체스
...,...,...,...
95,95,https://blog.naver.com/titi2010?Redirect=Log&l...,"맨유, 잭 그릴리쉬 영입 협상 그만 둘 수 있다"
96,96,https://blog.naver.com/sunsuboy2?Redirect=Log&...,맨유 웨스트햄과 무승부 첼시는 리버풀에 패배
97,97,https://blog.naver.com/peace577?Redirect=Log&l...,맨유로 이적한 오바메양..
98,98,https://2003ehgus.blog.me/221994918065,[Metro] 센터백 테덴 멩기는 맨유 1군으로 콜업되었습니다


In [ ]:
dict = {}  # 전체 크롤링 데이터를 담을 그릇

number = 100 # 수집할 글 갯수 (엑셀 파일에서 글 갯수와 비교해서 자신이 수집할 갯수를 선정)
for i in tqdm(range(0, number)): 
    # 글 띄우기
    url = url_load['url'][i]   # url은 엑셀에서 url 이라는 제목을 기준으로 사이트를 크롤링했고 이를 다시 띄우기 위해 사용 
    driver = webdriver.Chrome("C:/Users/USER/Desktop/code/chromedriver")
    driver.get(url)   # 글 띄우기
    
    # 사이트에서 글 크롤링 try, except는 100개라는 크롤링을 하던 중 일부 진행하다가 error가 날 경우를 대비하기 위해서 사용 
    try : 
        # iframe 접근
        driver.switch_to_frame('mainFrame')
        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-fs-.se-ff-" # 제목이 가지고 있는 사이트 html의 element                                 
        tit = driver.find_element_by_css_selector(overlays)  # 제목이 가지고 있는 사이트 element   
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"         # 글쓴이가 가지고 있는 사이트 html의 element
        nick = driver.find_element_by_css_selector(overlays)         
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"      # 날짜가 가지고 있는 사이트 html의 element                           
        date = driver.find_element_by_css_selector(overlays)        
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"    # 내용이 가지고 있는 사이트 html의 element                                
        contents = driver.find_elements_by_css_selector(overlays)        

        content_list = []             # 내용의 경우 블로그 마다 장문으로 여러 줄로 이루어져있기에 리스트를 만들어서 하나로 합치는 작업을 해야함
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)    # .join은 문장을 하나로 합치는 함수

        # 전체 100개의 게시글 중에 글 하나를 target_info라는 딕셔너리에 담고 이를 전체적으로 dict라는 딕셔너리에 담는 다
        # target_info 딕셔너리에 title ~ content라는 key 값으로 데이터들이 담김
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글을 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)  # time.sleep을 해주면서 웹 브라우저를 켜고 닫는 순간을 나누기 위해서 사용
        
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기 (for문을 반복하여 크롤링시 새로운창이 계속 뜨게 되기에 닫아준다)
        driver.close()       
    
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 글 30, 50, 80 번째 마다 엑셀로 저장 (데이터 중간, 중간에 파일로 저장하기 위해서 함)
    if i == 30 or 50 or 80:
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_내용 크롤링.xlsx")
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)# 수집한 글 개수 확인

result_df = pd.DataFrame.from_dict(dict, 'index')

# 저장하기
result_df.to_excel("blog_내용 크롤링.xlsx")

<ipython-input-6-28118a5aadcc>:13: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('mainFrame')


1 썸네일
2 라울 히메네스, 그는 맨유에 어울리는 공격수일까? 
3 머쓱
4 맨유를 떠나 인터밀란으로 이적하는 알렉시스 산체스
5 페란 토레스 맨시티 이적, 스몰링 임대 종료 맨유 복귀, 아르샤빈 코치, FC서울 김호영 감독대행 선임
6 [19-20 시즌 리뷰] 맨유 드디어 챔피언스리그로 복귀하다!
7 19-20 프리미어리그 38R 맨유 레스터, 첼시 울버햄튼, 레스터 맨유, 토트넘 크리스탈 팰리스 중계 프리뷰(feat. 손흥민 골 토트넘 유로파리그 가자!!)
8 [EPL 프리뷰] 남은 일정이 수월한 맨체스터 유나이티드, 챔피언스리그로 가나! 프리미어리그 맨유 대 본머스
9 유로파리그 16강 맨유 v LASK 프리뷰, 2군으로 상대할 예정?
10 라울 히메네스
12 여러 매체들이 ESPN에 보도한 바에 의하면, 보루시아 도르트문트가 맨체스터 유나이티드를 제치고 2003년생 미드필더 주드 벨링엄(Jude Bellingham, 16)의 영입에 성공했습니다. 
14 (英 후스코어드)
15 4대 미드필더 맨유레전드 데이비드베컴에 대해알아보자
16 안녕하세요 이웃님들!^^ 지난 치열했던 경기 살펴보셨나요?
17 아디다스 골프 스파이크리스 맨유 스페셜에디션 특별해서 좋아
18 맨유와 린츠의 유로파리그 16강 2차전 프리뷰 (맨체스터 유나이티드 LASK)
19 
20 [유로파리그 8월6일] 맨유 LASK린츠 / 인터밀란 헤타페
21 [EPL 프리뷰] 연승 끊긴 맨유, 동기부여 잃은 크리스탈팰리스를 만나다! (크팰 대 맨유_SPOTV중계)
22 [외신] 인테르 완전 이적 앞둔 산체스, 맨유에선 왜 실패했나
23 [선수 소개] 맨유, 풀백 걱정은 이제 그만 "알바로 페르난데스 카레라스"
24 2019-20 유로파리그 16강 2차전
25 이글은 읽으시면 더이상은 혼자서 힘들어 하시지 않아도 됩니다.
27 [ 주간 맨유 ] 제이든 산초 5년 계약 합의 근접  & 산체스 인테르 완전 이적 임박
28 epl의 강호  맨유 레전드들을  알아보자
29 
31 제시 린가드 극장골로 맨유